In [ ]:
pip install numpy pandas matplotlib seaborn bokeh selenium


In [ ]:
import bokeh
import pandas as pd

from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.io import output_notebook
import pandas as pd
from bokeh.models import Legend
from bokeh.palettes import Spectral, brewer, Viridis256, Spectral11, Viridis
from bokeh.palettes import brewer
from bokeh.models import ColumnDataSource, ColorBar
from bokeh.transform import linear_cmap
from bokeh.resources import INLINE
import bokeh.io
from bokeh import *
bokeh.io.output_notebook(INLINE)

from bokeh.io import output_notebook, export_svgs, save, reset_output
from bokeh import palettes
from bokeh.models import Legend
from bokeh.palettes import Spectral
from bokeh.palettes import brewer

In [ ]:
x1 = pd.ExcelFile("Report.xlsx")
x1.sheet_names

['Clean typology2', 'NISQ', 'Φύλλο4']

In [ ]:
df_main = pd.read_excel("Report.xlsx", sheet_name="NISQ")
df_main.head()

,Algorithms,Main subroutine,Fundamental Math Problem,Application,Unnamed: 4,Unnamed: 5,Unnamed: 6,NISQ / LSQ,Dates,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,QFT,Not-Applicable,Hidden subgroup problem. Dynamic system: Fouri...,NaN,NaN,NaN,NaN,LSQ,20/11/1995,DYS,HSP,NaN,NaN
1,QPE,QFT,Linear algebra: Phase estimation of an eigenve...,First-principle quantum simulation,FQS,NaN,NaN,LSQ,20/11/1995,LA,DYS,NaN,NaN
2,Shor,QPE,"Hidden subgroup problem: Factorization, Discre...","Crytanalysis-Asymmetric (Breaking RSA, Diffie-...",CR,NaN,NaN,LSQ,30/08/1995,HSP,NaN,NaN,NaN
3,HHL,"QPE, Sparse HS",Linear algebra: Solving systems for linear equ...,"Machine learning & Data science, First-princip...",DS,FQS,OP,LSQ,19/11/2018,LA,NaN,NaN,NaN
4,VQE,Not-Applicable,Linear algebra: Estimate phase of a unitary op...,First-principle quantum simulation: (https://a...,FQS,OP,NaN,NISQ,10/04/2013,LA,DYS,NaN,NaN


In [ ]:
df_relevent = df_main[["Algorithms", "NISQ / LSQ "] + [f"Unnamed: {i}" for i in range(9, 13)]]
df_relevent

,Algorithms,NISQ / LSQ,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,QFT,LSQ,DYS,HSP,NaN,NaN
1,QPE,LSQ,LA,DYS,NaN,NaN
2,Shor,LSQ,HSP,NaN,NaN,NaN
3,HHL,LSQ,LA,NaN,NaN,NaN
4,VQE,NISQ,LA,DYS,NaN,NaN
...,...,...,...,...,...,...
140,Q. Full configuration interaction (FCI),NISQ,DYS,NaN,NaN,NaN
141,Q. Density funtional theory(DFT) method,LSQ,DYS,NaN,NaN,NaN
142,Circuit formulation of adiabatic graphene simu...,LSQ,DYS,NaN,NaN,NaN
143,Variational protein folding simulation,NISQ,DYS,NaN,NaN,NaN


In [ ]:
df_relevent['Algorithms'][~df_relevent['Algorithms'].isna()]

0                                                    QFT
1                                                    QPE
2                                                   Shor
3                                                    HHL
4                                                    VQE
                             ...                        
140              Q. Full configuration interaction (FCI)
141              Q. Density funtional theory(DFT) method
142    Circuit formulation of adiabatic graphene simu...
143               Variational protein folding simulation
144                 Q. algo. for alchemical optimization
Name: Algorithms, Length: 129, dtype: object

In [ ]:
# we dont need these rows

df_relevent.iloc[145:, :]

,Algorithms,NISQ / LSQ,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12


In [ ]:
df_relevent.loc[:, "Math"] = df_relevent.iloc[:, 2:].apply(lambda row: {v for v in row.values if isinstance(v, str)}, axis=1)

<ipython-input-43-a64a19f209e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevent.loc[:, "Math"] = df_relevent.iloc[:, 2:].apply(lambda row: {v for v in row.values if isinstance(v, str)}, axis=1)


In [ ]:
df_relevent = df_relevent.iloc[:145, :][["NISQ / LSQ ", "Algorithms", "Math"]]
df_relevent

,NISQ / LSQ,Algorithms,Math
0,LSQ,QFT,"{HSP, DYS}"
1,LSQ,QPE,"{LA, DYS}"
2,LSQ,Shor,{HSP}
3,LSQ,HHL,{LA}
4,NISQ,VQE,"{LA, DYS}"
...,...,...,...
140,NISQ,Q. Full configuration interaction (FCI),{DYS}
141,LSQ,Q. Density funtional theory(DFT) method,{DYS}
142,LSQ,Circuit formulation of adiabatic graphene simu...,{DYS}
143,NISQ,Variational protein folding simulation,{DYS}


In [ ]:
df_segmented = df_relevent\
            .explode("Math")\
            .dropna(subset=['Math'])\
            .groupby("Math")\
            .agg({"NISQ / LSQ ": lambda x: {cat: x.tolist().count(cat) for cat in set(x.tolist())}})\
            ['NISQ / LSQ '].apply(pd.Series)

df_segmented.loc[:, "total"] = df_segmented.sum(axis=1)
df_segmented.loc[:, "LSQ"] = df_segmented["LSQ"]/df_segmented["total"]
df_segmented.loc[:, "NISQ"] = df_segmented["NISQ"]/df_segmented["total"]
df_segmented.drop(columns=["total"], inplace=True)
df_segmented

,NISQ,LSQ
Math,,
COM,0.433333,0.566667
DYS,0.479167,0.520833
HSP,0.153846,0.846154
LA,0.500000,0.500000
OP,0.230769,0.769231
SP,1.000000,NaN
ST,0.266667,0.733333


In [ ]:
df_segmented.reset_index(inplace=True)

In [ ]:
df_segmented.head()

,Math,NISQ,LSQ
0,COM,0.433333,0.566667
1,DYS,0.479167,0.520833
2,HSP,0.153846,0.846154
3,LA,0.500000,0.500000
4,OP,0.230769,0.769231


In [ ]:
df_segmented.columns

Index(['Math', 'NISQ', 'LSQ'], dtype='object')

In [ ]:
# Define the desired order for the domains
#desired_domain_order = ['ML', 'DS', 'BI&CI','QI&QT', 'OP','CR', 'FCP', 'FCC', 'CCP']

LSQ = df_segmented[["Math", "LSQ"]]
NISQ = df_segmented[["Math", "NISQ"]]
LSQ = {key: [v for v in value.values()] for key, value in LSQ.to_dict().items()}
NISQ = {"LSQ" if key == "NISQ" else key: [v for v in value.values()] for key, value in NISQ.to_dict().items()}
domains = LSQ['Math']
# Filter and reorder the domains list based on desired order
#domains = [domain for domain in desired_domain_order if domain in LSQ['applications']]
domains

['COM', 'DYS', 'HSP', 'LA', 'OP', 'SP', 'ST']

In [ ]:
classes = ["LSQ"]
LSQ = {'domains' : domains,
           'LSQ'   : LSQ["LSQ"]
          }

NISQ = {'domains' : domains,
           'LSQ'   : NISQ["LSQ"]
          }

LSQ['LSQ'] = [i*100 for i in LSQ['LSQ']]
NISQ['LSQ'] = [-i*100 for i in NISQ['LSQ']]

In [ ]:
pip install --upgrade bokeh


In [ ]:
from bokeh.models import FuncTickFormatter

In [58]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

output_notebook()

p = figure(y_range=domains,
           height=350,
           x_range=(-105, 105),
           title="")

p.hbar_stack(classes,
             y='domains',
             height=0.9,
             color='teal',
             source=ColumnDataSource(NISQ),
             legend_label=["NISQ"])

p.hbar_stack(classes,
             y='domains',
             height=0.9,
             color='darkseagreen',
             source=ColumnDataSource(LSQ),
             legend_label=["LSQ"])

p.title.align = "center"
p.title.text_font_style = "italic"
p.title.text_font_size = '18pt'
p.y_range.range_padding = 0.1
p.ygrid.grid_line_color = None
p.legend.location = "bottom_right"
p.axis.minor_tick_line_color = None
p.outline_line_color = None

p.xaxis.axis_label = 'Percentage (%)'
p.yaxis.axis_label = 'Mathematical Problem Category'
p.xaxis.axis_label_text_font_size = '14pt'
p.yaxis.axis_label_text_font_size = '14pt'

p.xaxis.major_label_text_font_size = '10pt'
p.yaxis.major_label_text_font_size = '10pt'
p.xaxis.major_label_text_font_style = 'italic'
p.yaxis.major_label_text_font_style = 'italic'

p.xaxis.major_label_overrides = {-50: '50', -100: '100'}
# Create a dictionary to map the original domain names to your desired labels
domain_labels = {
    'OP': 'Optimization',
    'COM': 'Combinatorial',
    'LA': 'Linear algebra',
    'HSP': 'Hidden-subgroup problem',
    'DYS': 'Dynamic system',
    'SP': 'Stochastic process',
    'ST': 'Statistics'
}

# Create a custom tick formatter for the y-axis
tick_formatter = FuncTickFormatter(code="""
    const labels = %s;
    return labels[tick] || tick;
""" % domain_labels)

p.yaxis.formatter = tick_formatter

# Apply the plot sizing using the sizing_mode attribute
p.sizing_mode = "fixed"  # You can also use "stretch_width" or "stretch_both" based on your needs
p.width = 800
p.height = 400

output_file(filename="stacked_lsq_nisq_math.html", title="nisq_lsq / Math")

show(p)

In [ ]:
from bokeh.colors import named

# Get the dictionary of named colors
all_named_colors = named.__dict__

# Print all the color names and their values
for color_name, color_value in all_named_colors.items():
    if not color_name.startswith('_'):  # Filter out private attributes
        print(f"{color_name}: {color_value}")


annotations: _Feature((3, 7, 0, 'beta', 1), (3, 11, 0, 'alpha', 0), 16777216)
logging: <module 'logging' from '/usr/lib/python3.10/logging/__init__.py'>
log: <Logger bokeh.colors.named (WARNING)>
NamedColor: <class 'bokeh.colors.util.NamedColor'>
aliceblue: aliceblue
antiquewhite: antiquewhite
aqua: aqua
aquamarine: aquamarine
azure: azure
beige: beige
bisque: bisque
black: black
blanchedalmond: blanchedalmond
blue: blue
blueviolet: blueviolet
brown: brown
burlywood: burlywood
cadetblue: cadetblue
chartreuse: chartreuse
chocolate: chocolate
coral: coral
cornflowerblue: cornflowerblue
cornsilk: cornsilk
crimson: crimson
cyan: cyan
darkblue: darkblue
darkcyan: darkcyan
darkgoldenrod: darkgoldenrod
darkgray: darkgray
darkgreen: darkgreen
darkgrey: darkgrey
darkkhaki: darkkhaki
darkmagenta: darkmagenta
darkolivegreen: darkolivegreen
darkorange: darkorange
darkorchid: darkorchid
darkred: darkred
darksalmon: darksalmon
darkseagreen: darkseagreen
darkslateblue: darkslateblue
darkslategray: da